# Download

In [ ]:
!wget -O gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg "https://zenodo.org/records/15631655/files/gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg?download=1"

In [ ]:
import os
import requests

DATA_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data"
os.makedirs(DATA_PATH, exist_ok=True)

url = "https://zenodo.org/records/15631655/files/gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg?download=1"

filename = os.path.join(DATA_PATH, "gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg")

response = requests.get(url, stream=True)
response.raise_for_status()

with open(filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Download concluído: {filename}")

# Extração

In [1]:
from pathlib import Path

BATCH_SIZE = 200_000

input_fn = "gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg"
input_dir = Path("../data")
output_dir = Path("../data/raw")
output_dir.mkdir(exist_ok=True)

In [ ]:
import geopandas as gpd

map_dict = {
    "Other land cover": 0.0,
    "Cultivated grassland": 2.0**6,
    "Natural/semi-natural grassland": 2.0**12,
    "Open shrubland": 2.0**18
}

processed = 0

while True:
    gdf = gpd.read_file(input_dir / input_fn, rows=slice(processed, processed + BATCH_SIZE))

    if gdf.empty:
        print("No more data to process.")
        break

    if gdf.shape[0] == BATCH_SIZE:
        last_tile_id = gdf.iloc[-1]["tile_id"]
    else:
        last_tile_id = None

    for tile_id in gdf["tile_id"].unique():
        if last_tile_id == tile_id:
            continue 

        tile_gdf = gdf[gdf["tile_id"] == tile_id].copy()

        tile_gdf["bing_value"] = tile_gdf["bing_class"].map(map_dict)
        tile_gdf["google_value"] = tile_gdf["google_class"].map(map_dict)
        
        tile_gdf.to_file(output_dir / f'{tile_id}.gpkg', driver="GPKG")

    processed += len(gdf)

    print(f"Processed up to rowid: {processed}")

Processed up to rowid: 200000
Processed up to rowid: 400000
Processed up to rowid: 600000
Processed up to rowid: 800000
Processed up to rowid: 1000000
Processed up to rowid: 1200000
Processed up to rowid: 1400000
Processed up to rowid: 1600000
Processed up to rowid: 1800000
Processed up to rowid: 2000000
Processed up to rowid: 2200000
Processed up to rowid: 2400000
Processed up to rowid: 2600000
Processed up to rowid: 2800000
Processed up to rowid: 3000000
Processed up to rowid: 3200000
Processed up to rowid: 3400000
Processed up to rowid: 3600000
Processed up to rowid: 3800000
Processed up to rowid: 4000000
Processed up to rowid: 4200000
Processed up to rowid: 4400000
Processed up to rowid: 4600000
Processed up to rowid: 4800000
Processed up to rowid: 5000000
Processed up to rowid: 5200000
Processed up to rowid: 5400000
Processed up to rowid: 5600000
Processed up to rowid: 5800000
Processed up to rowid: 6000000
Processed up to rowid: 6200000
Processed up to rowid: 6400000
Processed up